## Running a bell circuit on different backends

In [33]:
import numpy as np
import time

from braket.circuits import Circuit
from braket.devices import LocalSimulator
from qiskit_braket_provider import AWSBraketProvider
from qiskit_braket_provider.providers import adapter
from braket.aws import AwsSession

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.primitives import BackendEstimator, Estimator
from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit.circuit import ParameterVector, Parameter

import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from braket_estimator import BraketEstimator

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")
AWSBraketProvider().backends()

[BraketBackend[Aria 1],
 BraketBackend[Aria 2],
 BraketBackend[Aspen-M-3],
 BraketBackend[Forte 1],
 BraketBackend[Harmony],
 BraketBackend[Lucy],
 BraketBackend[SV1],
 BraketBackend[dm1]]

In [96]:
n_shots = 10_000

#### Run with native Braket SV Backend

In [10]:
backend = AWSBraketProvider().get_backend('SV1')

device = backend._device
device

In [20]:
braket_bell_circ = Circuit().h(0).cnot(0, 1)
print(braket_bell_circ)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [24]:
start_time = time.time()
job_braket_native = device.run_batch(
    [braket_bell_circ] * batchsize,
    # inputs = [bound_parameters] * batchsize,
    shots=n_shots,
)
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
print('Execution time per batch:', round(run_batch_time / batchsize, 2), 'seconds')

--- 14.032721281051636 seconds ---
Execution time per batch: 0.28


#### Run with Braket Provider + **Estimator Primitive**

In [25]:
qiskit_bell_circ = QuantumCircuit(2)
qiskit_bell_circ.h(0)
qiskit_bell_circ.cx(0, 1)

qiskit_bell_circ.draw("mpl")

In [55]:
backend = AWSBraketProvider().get_backend('SV1')
estimator = BackendEstimator(backend)
qiskit_observables = SparsePauliOp.from_list(([('XX', 0.25), ("IZ", 0.25), ("ZZ", 0.25), ("XY", 0.25)]))

qiskit_start_time = time.time()
job = estimator.run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
job.result().values # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per batch:', round(qiskit_run_batch_time / batchsize, 2), 'seconds')

--- 511.51 seconds ---
Execution time per batch: 10.23 seconds


In [56]:
qiskit_expvals = job.result().values

#### Use ``Estimator`` from Qiskit

In [73]:
start_time = time.time()
job = Estimator().run(
    circuits=[qiskit_bell_circ] * batchsize,
    observables=[qiskit_observables] * batchsize,
    shots=n_shots
)
qiskit_expvals_2 = job.result().values
run_batch_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_batch_time, 2)))
print('Execution time per batch:', round(run_batch_time / batchsize, 2), 'seconds')

--- 0.02 seconds ---
Execution time per batch: 0.0 seconds


In [93]:
qiskit_expvals_2 - qiskit_expvals

array([ 5.24114113e-03, -7.88743167e-03,  5.08701419e-03,  1.76689323e-03,
       -9.63843671e-04,  1.68695896e-03, -2.58888419e-03, -6.66856608e-03,
        5.86150076e-04, -2.31681255e-03,  4.66755497e-03, -2.01877144e-03,
        5.28116699e-03, -8.93541060e-03, -3.14745355e-03,  2.49434169e-03,
       -2.84772076e-03,  4.38220672e-03,  2.50710385e-03,  2.97318855e-03,
        1.45903786e-02,  1.13617448e-04,  6.96880763e-03, -7.07208498e-03,
        1.10526030e-02, -2.10099309e-03,  1.95981763e-03, -1.34841878e-02,
       -6.13131788e-04, -2.78217183e-03, -1.56774213e-02,  9.59081887e-03,
       -2.41544184e-03, -4.88744183e-03, -2.40772335e-03, -1.58772304e-03,
       -1.16462414e-02,  9.25145759e-05,  2.65408283e-03,  9.74644321e-04,
        8.94780071e-03, -7.42534855e-03,  6.73199679e-03, -2.96371375e-03,
       -7.17149421e-04, -4.75080901e-03,  4.23577630e-03,  7.37570691e-03,
       -6.72249946e-03,  3.11197895e-03])

In [94]:
max(qiskit_expvals_2 - qiskit_expvals)

0.014590378597244824

#### Run Circuit and Measure observables on own ``BraketEstimator``

In [57]:
backend = LocalSimulator()
estimator = BraketEstimator(backend)

braket_observables = qiskit_observables.to_list()

In [58]:
start_time = time.time()
expvals = estimator.run(
    circuit=[braket_bell_circ] * batchsize,
    observables=[braket_observables] * batchsize,
    target_register=[[0, 1]] * batchsize,
    shots=n_shots
)
run_braket_estimator_time = time.time() - start_time
print('--- {} seconds ---'.format(round(run_braket_estimator_time, 2)))
print('Execution time per batch:', round(run_braket_estimator_time / batchsize, 2), 'seconds')

--- 5.04 seconds ---
Execution time per batch: 0.1 seconds


In [70]:
expvals.astype(float) - qiskit_expvals

/var/folders/6b/76tj41f506g7_0dg_k15w_d80000gn/T/ipykernel_11078/798129361.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  expvals.astype(float) - qiskit_expvals


array([ 3.250e-03,  6.000e-04,  7.500e-03,  6.000e-04,  6.550e-03,
       -2.450e-03, -5.500e-03, -1.215e-02, -1.140e-02, -1.250e-03,
        6.250e-03,  5.150e-03,  5.950e-03, -1.000e-02,  3.150e-03,
        7.550e-03, -9.550e-03, -5.250e-03,  1.750e-03, -1.500e-03,
        9.700e-03, -8.500e-04, -3.250e-03,  1.550e-03, -1.300e-03,
       -2.500e-04, -5.250e-03, -1.025e-02,  7.300e-03,  7.000e-04,
       -1.065e-02,  3.500e-03,  4.000e-04, -4.150e-03,  1.900e-03,
        6.150e-03, -1.100e-02, -1.500e-03,  1.800e-03, -4.500e-03,
        4.800e-03, -2.250e-03,  5.500e-04, -5.000e-04,  3.550e-03,
        2.600e-03, -1.700e-03,  3.900e-03, -5.000e-05, -4.000e-03])

In [69]:
max(expvals.astype(float) - qiskit_expvals)

/var/folders/6b/76tj41f506g7_0dg_k15w_d80000gn/T/ipykernel_11078/3208180139.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  max(expvals.astype(float) - qiskit_expvals)


0.009699999999999931

#### Running batch of qiskit circuits with SV1 backend provider

In [74]:
backend = AWSBraketProvider().get_backend('SV1')

In [86]:
qiskit_start_time = time.time()
job = backend.run(
    [qiskit_bell_circ] * batchsize,
    shots=n_shots
)
result = job.result() # This is the time-consuming line

qiskit_run_batch_time = time.time() - qiskit_start_time
print('--- {} seconds ---'.format(round(qiskit_run_batch_time, 2)))
print('Execution time per batch:', round(qiskit_run_batch_time / batchsize, 2), 'seconds')

--- 150.02 seconds ---
Execution time per batch: 3.0 seconds
